In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import requests
import os

# ====== Selenium 설정 ======
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=options)

# ====== 크롤링 대상 URL ======
url = "https://www.1388.go.kr/occ/YTOSP_SC_OID_02?bbs=10078&agency=U03&id=31200&flag=OID&page=1"
driver.get(url)
time.sleep(3)  # JS 렌더링 대기

soup = BeautifulSoup(driver.page_source, "html.parser")

# ✅ 제목 추출
title_tag = soup.select_one("tr.board-title td > div.cell")
title = title_tag.get_text(strip=True) if title_tag else "제목 없음"

# ✅ 작성일 추출
date_tag = soup.select_one("tr.board-info td > div.cell")
date = date_tag.get_text(strip=True) if date_tag else "작성일 없음"

# ✅ 본문 추출
content_tag = soup.select_one("tr.board-content span")
content = content_tag.get_text("\n", strip=True) if content_tag else "본문 없음"

# ✅ 이미지 다운로드 (로컬 저장)
img_tag = soup.select_one("tr.board-content img")
if img_tag and "src" in img_tag.attrs:
    img_url = img_tag["src"]
    if img_url.startswith("/"):
        img_url = "https://www.1388.go.kr" + img_url

    # 이미지 저장 경로 설정
    img_filename = f"{title[:30].replace(' ', '_')}.jpg"
    img_path = os.path.join(os.getcwd(), img_filename)

    # 다운로드 및 저장
    img_data = requests.get(img_url).content
    with open(img_path, "wb") as f:
        f.write(img_data)
else:
    img_url = "이미지 없음"
    img_path = None

driver.quit()

# ✅ 결과 출력
print("제목:", title)
print("작성일:", date)
print("본문:\n", content)
print("이미지 URL:", img_url)
if img_path:
    print(f"✅ 이미지가 로컬에 저장되었습니다: {img_path}")
else:
    print("❌ 이미지 없음")


제목: 2026학년도 학교 밖 청소년 대학입시설명회 및 컨설팅 안내
작성일: 2025-04-23
본문:
 안녕하세요, 한국청소년상담복지개발원입니다.
전국 꿈드림센터에서는 대학 입시를 희망하는 학교 밖 청소년을 위해 대학입시설명회, 맞춤형 컨설팅을 지원하고 있습니다.
관심있는 청소년은 근처
꿈드림센터로 문의
바랍니다. 감사합니다.
첨부파일에서 지역별 일정표와 연락처를 확인해보세요♬
이미지 URL: https://gov.youthsafety.go.kr/upfile/isry_backend/U16/MAIN/2025/4/23/2025-04-23_208f5427-ee8d-4a32-9bc1-34374fa88c70.jpg
✅ 이미지가 로컬에 저장되었습니다: c:\Users\jrnee\Desktop\babybaby\2026학년도_학교_밖_청소년_대학입시설명회_및_컨설팅.jpg


<h1>청소년 1388 크롤링 사이트

In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import requests
import os
import csv

# 🔗 크롤링할 게시글 URL 목록
urls = [
    "https://www.1388.go.kr/occ/YTOSP_SC_OID_02?bbs=10078&agency=U03&id=31200&flag=OID&page=1",
    "https://www.1388.go.kr/occ/YTOSP_SC_OID_02?bbs=10078&agency=U03&id=31046&flag=OID&page=1",
    "https://www.1388.go.kr/occ/YTOSP_SC_OID_02?bbs=10078&agency=U03&id=29325&flag=OID&page=1",
    "https://www.1388.go.kr/occ/YTOSP_SC_OID_02?bbs=10078&agency=U03&id=29307&flag=OID&page=1",
    "https://www.1388.go.kr/occ/YTOSP_SC_OID_02?bbs=10078&agency=U03&id=29130&flag=OID&page=1",
    "https://www.1388.go.kr/occ/YTOSP_SC_OID_02?bbs=10078&agency=U03&id=29110&flag=OID&page=1",
    "https://www.1388.go.kr/occ/YTOSP_SC_OID_02?bbs=10078&agency=U03&id=29084&flag=OID&page=1",
    "https://www.1388.go.kr/occ/YTOSP_SC_OID_02?bbs=10078&agency=U03&id=29063&flag=OID&page=1",
    "https://www.1388.go.kr/occ/YTOSP_SC_OID_02?bbs=10078&agency=U03&id=28967&flag=OID&page=1",
    "https://www.1388.go.kr/occ/YTOSP_SC_OID_02?bbs=10078&agency=U03&id=28965&flag=OID&page=1"
]

# 📁 저장 폴더
output_dir = "1388_output"
os.makedirs(output_dir, exist_ok=True)

# 💾 CSV 저장 파일
csv_path = os.path.join(output_dir, "1388_공지_크롤링.csv")

# 🧭 Selenium 설정
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

# 📊 CSV 작성
with open(csv_path, mode="w", newline="", encoding="utf-8-sig") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["제목", "작성일", "본문", "이미지 저장 경로", "PDF 저장 경로", "게시글 링크"])

    for url in urls:
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # 제목
        title_tag = soup.select_one("tr.board-title td > div.cell")
        title = title_tag.get_text(strip=True) if title_tag else "제목 없음"
        safe_title = title[:30].replace(" ", "_").replace("/", "_")

        # 작성일
        date_tag = soup.select_one("tr.board-info td > div.cell")
        date = date_tag.get_text(strip=True) if date_tag else "작성일 없음"

        # 본문
        # 본문 추출 - span과 div.cell 모두 대응
        # 본문 추출 개선 (줄바꿈 처리 포함)
        content_tag = soup.select_one("tr.board-content span") or soup.select_one("tr.board-content div.cell")

        if content_tag:
            # 줄바꿈 태그 <br> 처리
            for br in content_tag.find_all("br"):
                br.replace_with("\n")

            # <p>, <li> 등 태그 사이도 줄바꿈 되도록
            texts = []
            for elem in content_tag.descendants:
                if elem.name in ["p", "li"]:
                    text = elem.get_text(separator=" ", strip=True)
                    if text:
                        texts.append(text)
                elif isinstance(elem, str):
                    clean = elem.strip()
                    if clean:
                        texts.append(clean)

            # 줄바꿈 포함된 텍스트로 본문 정리
            content = "\n".join(texts)
        else:
            content = "본문 없음"

        # 이미지 다운로드
        img_tag = soup.select_one("tr.board-content img")
        img_path = ""
        if img_tag and "src" in img_tag.attrs:
            img_url = img_tag["src"]
            if img_url.startswith("/"):
                img_url = "https://www.1388.go.kr" + img_url
            img_path = os.path.join(output_dir, f"{safe_title}.jpg")
            try:
                img_data = requests.get(img_url).content
                with open(img_path, "wb") as f:
                    f.write(img_data)
            except:
                img_path = ""

        # PDF 다운로드
        pdf_tags = soup.select('a[href$=".pdf"]')
        pdf_path = ""
        for pdf_tag in pdf_tags:
            relative = pdf_tag["href"]
            if relative.startswith("/"):
                pdf_url = "https://www.1388.go.kr" + relative
                pdf_path = os.path.join(output_dir, relative.split("/")[-1])
                try:
                    pdf_data = requests.get(pdf_url).content
                    with open(pdf_path, "wb") as f:
                        f.write(pdf_data)
                except:
                    pdf_path = ""
                break  # 첫 번째 PDF만 다운로드

        # CSV 저장
        writer.writerow([title, date, content, img_path, pdf_path, url])
        print(f"✅ 저장 완료: {title}")

driver.quit()
print(f"\n📁 모든 게시글 정보가 CSV로 저장되었습니다: {csv_path}")


✅ 저장 완료: 2026학년도 학교 밖 청소년 대학입시설명회 및 컨설팅 안내
✅ 저장 완료: 2025년 학교 밖 청소년 지원 사업 홍보 콘텐츠 공모전 안내
✅ 저장 완료: [기간연장] 학교 밖 청소년 부모교육 콘서트 「달라도 좋은 우리!」 신청 안내
✅ 저장 완료: [이벤트]고립·은둔 청소년에게 희망과 용기를! #밖으로챌린지
✅ 저장 완료: 2024 학교 밖 청소년 Y.E.S! 꿈드림 축제를 소개해요
✅ 저장 완료: 학교 밖 청소년, 건강검진 수검 이벤트
✅ 저장 완료: 학교 밖 청소년 학습이력관리 시스템 '다이어DREAM' 수강인증 이벤트
✅ 저장 완료: 청소년생활기록부를 소개합니다!
✅ 저장 완료: [소개해요] 학교 밖 청소년 창업기업 안내
✅ 저장 완료: [소개해요] 2024년 학교 밖 청소년 지원 안내서

📁 모든 게시글 정보가 CSV로 저장되었습니다: 1388_output\1388_공지_크롤링.csv


In [30]:
import pandas as pd
import os

# 📁 경로 설정
output_dir = "1388_output"
csv_path = os.path.join(output_dir, "1388_공지_크롤링.csv")
xlsx_path = os.path.join(output_dir, "1388_공지_크롤링_변환.xlsx")

# ✅ CSV 읽기
df = pd.read_csv(csv_path, encoding='utf-8-sig')

# ✅ Excel 저장
df.to_excel(xlsx_path, index=False)

print(f"✅ 변환 완료: {xlsx_path}")


✅ 변환 완료: 1388_output\1388_공지_크롤링_변환.xlsx
